In [8]:

import streamlit as st

import networkx as nx
#import matplotlib.pyplot as plt 
#import pandas as pd
#import seaborn as sns
import logging
import geopandas as gpd

import osmnx as ox
import networkx as nx
#import plotly.graph_objects as go
import numpy as np
import pickle
import os

#import plotly.express as px

import fiona
from streamlit_folium import folium_static
import folium
from shapely.ops import nearest_points


from shapely.geometry import LineString
from shapely.geometry import Point


In [2]:
input_file = '../data/networkx_python_shp/shp_input_networkx_EPSG32648/split_intersection.shp'

In [3]:
G= nx.readwrite.nx_shp.read_shp(input_file, simplify=False)

In [26]:
nodes=True
edges=True
node_geometry=True
fill_edge_geometry=True
crs = 32648
if not G.nodes:  # pragma: no cover
    raise ValueError("graph contains no nodes")

nodes, data = zip(*G.nodes(data=True))
#logging.info(f'{nodes[0]}')
#logging.info(f'{data}')
#if node_geometry:
    # convert node x/y attributes to Points for geometry column
#[Point(d["x"], d["y"]) for d in data]
geom = [Point(n) for n in nodes]
x = [d[0] for d in nodes]
y = [d[1] for d in nodes]
data = {'x': x, 'y': y}
#geom = (n for n in nodes)
#     logging.info(f'{geom}')

gdf_nodes = gpd.GeoDataFrame(data, index=nodes, crs=crs, geometry=list(geom))
gdf_nodes.reset_index().drop(columns=['level_0','level_1'])
# else:
#     gdf_nodes = gpd.GeoDataFrame(data, index=nodes)

# gdf_nodes.index.rename("osmid", inplace=True)
# #utils.log("Created nodes GeoDataFrame from graph")

,,x,y,geometry
383523.677547,147619.944594,383523.677547,147619.944594,POINT (383523.678 147619.945)
383541.655113,147627.691646,383541.655113,147627.691646,POINT (383541.655 147627.692)
382844.644219,147322.573310,382844.644219,147322.573310,POINT (382844.644 147322.573)
382843.054840,147327.115010,382843.054840,147327.115010,POINT (382843.055 147327.115)
382834.079407,147307.240872,382834.079407,147307.240872,POINT (382834.079 147307.241)
...,...,...,...,...
383466.380582,146305.249148,383466.380582,146305.249148,POINT (383466.381 146305.249)
383478.873113,146313.442953,383478.873113,146313.442953,POINT (383478.873 146313.443)
383372.667922,146130.684333,383372.667922,146130.684333,POINT (383372.668 146130.684)
383382.197243,146121.421121,383382.197243,146121.421121,POINT (383382.197 146121.421)


In [34]:
gdf_nodes[['x','y']]

,,x,y
383523.677547,147619.944594,383523.677547,147619.944594
383541.655113,147627.691646,383541.655113,147627.691646
382844.644219,147322.573310,382844.644219,147322.573310
382843.054840,147327.115010,382843.054840,147327.115010
382834.079407,147307.240872,382834.079407,147307.240872
...,...,...,...
383466.380582,146305.249148,383466.380582,146305.249148
383478.873113,146313.442953,383478.873113,146313.442953
383372.667922,146130.684333,383372.667922,146130.684333
383382.197243,146121.421121,383382.197243,146121.421121


In [29]:
def nearest_nodes(G, X, Y, return_dist=False):
    """
    Find the nearest node to a point or to each of several points.
    If `X` and `Y` are single coordinate values, this will return the nearest
    node to that point. If `X` and `Y` are lists of coordinate values, this
    will return the nearest node to each point.
    If the graph is projected, this uses a k-d tree for euclidean nearest
    neighbor search, which requires that scipy is installed as an optional
    dependency. If it is unprojected, this uses a ball tree for haversine
    nearest neighbor search, which requires that scikit-learn is installed as
    an optional dependency.
    Parameters
    ----------
    G : networkx.MultiDiGraph
        graph in which to find nearest nodes
    X : float or list
        points' x (longitude) coordinates, in same CRS/units as graph and
        containing no nulls
    Y : float or list
        points' y (latitude) coordinates, in same CRS/units as graph and
        containing no nulls
    return_dist : bool
        optionally also return distance between points and nearest nodes
    Returns
    -------
    nn or (nn, dist) : int/list or tuple
        nearest node IDs or optionally a tuple where `dist` contains distances
        between the points and their nearest nodes
    """
    is_scalar = False
    if not (hasattr(X, "__iter__") and hasattr(Y, "__iter__")):
        # make coordinates arrays if user passed non-iterable values
        is_scalar = True
        X = np.array([X])
        Y = np.array([Y])

    if np.isnan(X).any() or np.isnan(Y).any():  # pragma: no cover
        raise ValueError("`X` and `Y` cannot contain nulls")
    nodes = graph_to_gdfs(G, edges=False, node_geometry=False)[["x", "y"]]

    if projection.is_projected(G.graph["crs"]):
        # if projected, use k-d tree for euclidean nearest-neighbor search
        if cKDTree is None:  # pragma: no cover
            raise ImportError("scipy must be installed to search a projected graph")
        dist, pos = cKDTree(nodes).query(np.array([X, Y]).T, k=1)
        nn = nodes.index[pos]

    else:
        # if unprojected, use ball tree for haversine nearest-neighbor search
        if BallTree is None:  # pragma: no cover
            raise ImportError("scikit-learn must be installed to search an unprojected graph")
        # haversine requires lat, lng coords in radians
        nodes_rad = np.deg2rad(nodes[["y", "x"]])
        points_rad = np.deg2rad(np.array([Y, X]).T)
        dist, pos = BallTree(nodes_rad, metric="haversine").query(points_rad, k=1)
        dist = dist[:, 0] * EARTH_RADIUS_M  # convert radians -> meters
        nn = nodes.index[pos[:, 0]]

    # convert results to correct types for return
    nn = nn.tolist()
    dist = dist.tolist()
    if is_scalar:
        nn = nn[0]
        dist = dist[0]

    if return_dist:
        return nn, dist
    else:
        return nn


In [31]:
graph_to_gdfs(G, edges=False, node_geometry=False)[["x", "y"]]

NameError: name 'graph_to_gdfs' is not defined

In [ ]:

def graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True):
    """
    Convert a MultiDiGraph to node and/or edge GeoDataFrames.
    This function is the inverse of `graph_from_gdfs`.
    Parameters
    ----------
    G : networkx.MultiDiGraph
        input graph
    nodes : bool
        if True, convert graph nodes to a GeoDataFrame and return it
    edges : bool
        if True, convert graph edges to a GeoDataFrame and return it
    node_geometry : bool
        if True, create a geometry column from node x and y attributes
    fill_edge_geometry : bool
        if True, fill in missing edge geometry fields using nodes u and v
    Returns
    -------
    geopandas.GeoDataFrame or tuple
        gdf_nodes or gdf_edges or tuple of (gdf_nodes, gdf_edges). gdf_nodes
        is indexed by osmid and gdf_edges is multi-indexed by u, v, key
        following normal MultiDiGraph structure.
    """
    crs = 32648

    if nodes:
        if not G.nodes:  # pragma: no cover
            raise ValueError("graph contains no nodes")

        nodes, data = zip(*G.nodes(data=True))
        logging.info(f'{nodes[0]}')
        #logging.info(f'{data}')
        if node_geometry:
            # convert node x/y attributes to Points for geometry column
            geom = (Point(d["x"], d["y"]) for d in data)
            geom = (n for n in nodes)
            logging.info(f'{geom}')
            gdf_nodes = gpd.GeoDataFrame(data, index=nodes, crs=crs, geometry=list(geom))
        else:
            gdf_nodes = gpd.GeoDataFrame(data, index=nodes)

        gdf_nodes.index.rename("osmid", inplace=True)
        utils.log("Created nodes GeoDataFrame from graph